In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import gantt
import importlib
from distinctipy import distinctipy

In [3]:
# filename = '../ciment_quebec/solution/CQ/2020101_det.csv'
filename = './20201003.csv'

In [15]:
df = pd.read_csv(filename,sep=";")
t1 = df.loc[:,['Del',	'ArrDock',	'FinDock','Driver','instance','Order','Cust','Depot'	]]
t1['Task'] = t1.apply(lambda x:f"Load_{int(x.Del)}",axis=1)
t1['Op'] = t1.apply(lambda x:f"L{int(x.Order)}",axis=1)
t1.rename(columns={'ArrDock':'start','FinDock':'end'},inplace=True)
t2 = df.loc[:,['Del',	'StartDel','FinDel','Driver','instance','Order','Cust','Depot'	]]
t2['Task'] = t2.apply(lambda x:f"UnLoad_{int(x.Del)}",axis=1)
t2['Op'] = t2.apply(lambda x:f"U{int(x.Order)}",axis=1)
t2.rename(columns={'StartDel':'start','FinDel':'end'},inplace=True)
df = pd.concat([t1,t2])
importlib.reload(gantt)
df['start'] = df.apply(lambda x: gantt.time_to_date( x.instance,x.start),axis=1)
df['end'] = df.apply(lambda x: gantt.time_to_date( x.instance,x.end),axis=1)

# gantt.showGanttDel(df)
# df

In [32]:
import importlib
importlib.reload(gantt)
gantt.showGanttDepot(df)

In [41]:
import importlib
importlib.reload(gantt)
gantt.show(df)

In [8]:
import importlib
importlib.reload(gantt)
gantt.showGanttDriver(df)

In [82]:
# df = pd.read_csv(filename,sep=";")
# df['start'] = df.apply(lambda x: gantt.time_to_date( x.Date,x.ArrDock),axis=1)
# df['end'] = df.apply(lambda x: gantt.time_to_date( x.Date,x.FinDel),axis=1)
# stat1 = df.groupby(['ordreId','OrderDemand','delId','driverId','depotLoc']).agg({'delDemand':['sum'],'ArrivalTimeDel':['sum'],'early_tw':['sum']})
# stat2 = df.groupby(['driverId','depotLoc','delId','ordreId','OrderDemand']).agg({'ArrivalTimeDel':['sum'],'early_tw':['sum']})
# stat3 = df.groupby(['depotLoc','driverId','driverRank','delId','delDemand','ordreId','OrderDemand']).agg({'ArrivalTimeDel':['sum'],'early_tw':['sum']})

# stat2.to_excel('stats.xlsx','stat2')


# stat4 = df.groupby(['ordreId','OrderDemand']).agg({'delDemand':['sum']})
# stat4.columns = [x[0] for x in stat4.columns]
# stat4.reset_index(inplace=True)
# stat4['diff']= stat4['OrderDemand'] - stat4['delDemand']
# order_unserved = stat4[stat4['diff']!=0]
# if len(order_unserved) > 0:
#     order_unserved.to_csv('order_unserved.csv')
# order_unserved.head()
# print(f"{len(df['driverId'].unique())} drivers used")
# print(f"{len(df['depotLoc'].unique())} depots used")
